In [1]:
from _build import dataset_from_config
from dptb.utils.config import Config

In [2]:
config = {
    "root": "/root/nequip_data/",
    "dataset": "npz",
    "dataset_file_name": "/root/nequip_data/Si8-100K.npz",
    "key_mapping":{
        "pos":"pos",
        "atomic_numbers":"atomic_numbers",
        "kpoints": "kpoint",
        "pbc": "pbc",
        "cell": "cell",
        "eigenvalues": "eigenvalue"
    },
    "npz_fixed_field_keys": ["kpoint", "pbc"],
    "graph_field":["eigenvalues"],
    "chemical_symbols": ["Si", "C"],
    "r_max": 4.0,
    "oer_max": 3.0
}

config = Config(config=config)
# dataset: npz                                                                       # type of data set, can be npz or ase
# dataset_url: http://quantum-machine.org/gdml/data/npz/toluene_ccsd_t.zip           # url to download the npz. optional
# dataset_file_name: ./benchmark_data/toluene_ccsd_t-train.npz                       # path to data set file
# key_mapping:
#   z: atomic_numbers                                                                # atomic species, integers
#   E: total_energy                                                                  # total potential eneriges to train to
#   F: forces                                                                        # atomic forces to train to
#   R: pos                                                                           # raw atomic positions
# npz_fixed_field_keys:                                                              # fields that are repeated across different examples
#   - atomic_numbers

# chemical_symbols:
#   - H
#   - C

In [3]:
dataset = dataset_from_config(config=config, prefix="dataset")

In [4]:
from dptb.nn._sktb import SKTB
sktb = SKTB(
    basis={"Si":["3s", "3p", "p*", "s*"], "C":["2s","2p"]},
    onsite="strain",
    hopping="powerlaw",
    overlap=False
    )

In [5]:
from dptb.data.AtomicDataDict import with_edge_vectors, with_onsitenv_vectors

data = with_edge_vectors(dataset[0].to_dict())
data = with_onsitenv_vectors(data)

In [6]:
import torch
data["atomic_numbers"] = dataset.type_mapper.untransform(data["atom_types"])

In [7]:
data = sktb(data)

In [8]:
from dptb.nn._hamiltonian import SKHamiltonian

skh = SKHamiltonian(basis={"Si":["3s", "3p", "p*", "s*"], "C":["2s","2p"]})

In [9]:
skh(data)

torch.Size([8, 1, 1, 1])
torch.Size([8, 1, 1, 1])
torch.Size([8, 1, 3, 3])
torch.Size([8, 1, 3, 3])
torch.Size([8, 4, 1, 1])


RuntimeError: output with shape [8, 3] doesn't match the broadcast shape [8, 4, 8, 3]